In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import tensorflow as tf

In [4]:
df = pd.read_csv('/content/fraud_detection_dataset.csv')
df.dropna(inplace=True)

# Define features and target
X = df.drop(['id', 'text_sample', 'is_fraud'], axis=1)
y = df['is_fraud']

# Standardize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [5]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 100.00%


In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9816 - loss: 0.0635 - val_accuracy: 0.9983 - val_loss: 0.0047
Epoch 2/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9989 - loss: 0.0034 - val_accuracy: 0.9987 - val_loss: 0.0043
Epoch 3/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9991 - loss: 0.0027 - val_accuracy: 0.9979 - val_loss: 0.0054
Epoch 4/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9991 - loss: 0.0023 - val_accuracy: 0.9987 - val_loss: 0.0039
Epoch 5/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9994 - loss: 0.0017 - val_accuracy: 0.9987 - val_loss: 0.0038
Epoch 6/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9995 - loss: 0.0014 - val_accuracy: 0.9989 - val_loss: 0.0034
Epoch 7/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9996 - loss: 0.0013 - val_accuracy: 0.9984 - val_loss: 0.0038
Epoch 8/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9993 - loss: 0.0019 

In [11]:
model.save('/fraud_detection_model.h5')

In [12]:
loaded_model = load_model('/fraud_detection_model.h5')
sample_input = np.array([X_test[0]])  # Example with the first test sample
sample_prediction = loaded_model.predict(sample_input)
print("Sample Prediction:", "Fraud" if sample_prediction > 0.5 else "Not Fraud")

# Convert the .h5 model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
Sample Prediction: Not Fraud
Saved artifact at '/tmp/tmpzrcshsls'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 12), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  138091251104432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138091249651728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138091250570816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138091250575216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138091250580144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138091250579088: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [15]:
with open('/fraud_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Optional: Convert to C array (.h) for embedding in ESP32


In [20]:
loaded_model = load_model('/fraud_detection_model.h5')
sample_input = np.array([X_test[50]])  # Example with the first test sample
print( "printing the input",sample_input)
sample_prediction = loaded_model.predict(sample_input)
print("Sample Prediction:", "Fraud" if sample_prediction > 0.5 else "Not Fraud")

printing the input [[-1.40008606  0.04910746  0.1432816   1.2376829   0.97362367 -0.79722532
  -1.50596202  0.19404412  0.94825123 -1.34629621  0.77663294 -0.04766371]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
Sample Prediction: Not Fraud


In [21]:
print( X_test)

[[-0.26227328  0.5250795  -0.79028857 ... -1.40770843  0.50668807
   0.82262582]
 [-1.312562    0.09670467 -0.62554089 ... -0.94711678  0.39099741
  -1.07153375]
 [ 0.65672934 -1.23601705  1.35143124 ... -0.08734568 -1.30579894
  -0.86675974]
 ...
 [ 1.00682558  0.09670467  0.58260874 ...  0.09689098 -1.07441762
  -0.76437273]
 [-1.09375185  0.28709348 -0.35096143 ... -0.05663957 -0.264583
   1.02739983]
 [-1.09375185 -0.76004501 -1.11978392 ... -1.65335732  0.8151965
   0.82262582]]


In [25]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/fraud_detection_model.h5')

# Get the indices of the first 100 samples in the original test set
first_100_indices = df.iloc[y_test.index[:100]].index

# Take the first 100 samples from the scaled X_test
X_test_sample = X_test[:100]

# Retrieve the corresponding text samples for the first 100 samples
text_samples = df['text_sample'].iloc[first_100_indices]

# Get predictions for the first 100 samples
predictions = loaded_model.predict(X_test_sample)

# Convert predictions to "Fraud" or "Not Fraud" labels and print results with text
results = ["Fraud" if pred > 0.5 else "Not Fraud" for pred in predictions]

for i, (text, result) in enumerate(zip(text_samples, results)):
    print(f"Sample {i + 1}: {result} - Text: {text}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Sample 1: Not Fraud - Text: Increased focus on operational efficiency has resulted in lower costs this year.
Sample 2: Not Fraud - Text: We continue to face challenges in revenue growth due to market conditions.
Sample 3: Not Fraud - Text: The recent acquisition has positively impacted our quarterly figures.
Sample 4: Not Fraud - Text: Increased focus on operational efficiency has resulted in lower costs this year.
Sample 5: Not Fraud - Text: In the past quarter, our financial performance has exceeded expectations.
Sample 6: Not Fraud - Text: In the past quarter, our financial performance has exceeded expectations.
Sample 7: Not Fraud - Text: Management remains optimistic despite minor downturns in specific sectors.
Sample 8: Not Fraud - Text: We anticipate stable growth, though external factors may introduce uncertainty.
Sample 9: Not Fraud - Text: Management remains optimistic despite minor downturns in specific sectors.
Sample 10: Not Fraud - Te